In [12]:
import feather as ft
import os
import pandas as pd
import numpy as np
import time
from sqlalchemy import create_engine
import datetime as dt
import re
import utils
import warnings
warnings.filterwarnings('ignore')

In [13]:
mapToGameSummary = {
    'aggame':['AsiaGameAgin', 'AsiaGameHunter', 'AsiaGameXin', 'AsiaGameYoplay'],
    'gdcasino':['GoldDeluxCasino'], 
    'immwgfish':['IMMegaWinFish'], 
    'keno':['LibiancKeno'],
    'grandsuite':['NewCasino188'],
    'sportsbook':['NewSport188'], 
    'owsport':['OWSportNumberGame', 'OWVirtualSport', 'OWSport'],
    'PG':['PGSlot'],
    'blcard':['BLCard'],
    'bwesport':['BWESport'],
    'vrlotto':['VRLotto'],
    'tcglotto':['TCGLotto'],
    'ptslot':['PTSlot'],
    'ppslot':['PPSlot'],
    'msclub':['MSCasino'],
    'keno':['LibiancKeno'],
    'kgkeno':['KGKeno'],
    'imsport':['IMSport', 'IMESport'],
    'imsglotto':['IMSGLotto'],
    'imsgcard':['IMSGCard'],
    'cqslot':['CQSlot'],
    'ebetcasino':['EBETCasino'],
    'hlcard':['HLCard'],
    'imcasino':['IMCasino'],
    'imkycard':['IMKYCard'],
    'imlycard':['IMLYCard'],
    'immtcard':['IMMTCard']
    }

In [14]:
path = '../../share/data_prod/'
# gameSummary = ft.read_dataframe(path + 'game_daily_summary.ft')
member_profile = ft.read_dataframe(path + 'member_profile.ft')
gameAll = ft.read_dataframe(path + 'game_all_in_one_begin20191101.ft').query('BetTime >= "2020-02-15"')
# gameAllFull = gameAll.copy()
# gameAll = gameAll.query('BetTime < "2020-03-02"')

In [29]:
# 重跑到哪天為止（不含）
SpecifyDate = '2020-03-07'

In [191]:
pathProd = '../../share/data_prod/'
member = ft.read_dataframe(pathProd + 'member.ft')

In [30]:
pathProd = '../../share/data_prod/'
pathMemberWallet = '../../share/data_prod/MemberWallets/'
pathMemberAccountLog = '../../share/data_prod/MemberAccountLog/'

In [31]:
member_profile = ft.read_dataframe(pathProd + 'member_profile.ft')
# 所有遊戲流水（為了處理『獲取地方錢包金額前下注，且於獲取地方錢包後派彩的問題』）
gameAll = ft.read_dataframe(pathProd + 'game_all_in_one_begin20191101.ft').query('PayoutTime >= "2020-02-15" & PayoutTime < @SpecifyDate')
# 玩家地方錢包資料（已處理新註冊用戶的資料）
MemberWallets = ft.read_dataframe(pathMemberWallet + 'rawMemberWallets.ft')

In [32]:
# MemberWallets = ft.read_dataframe(pathMemberWallet + 'MemberWalletsFlow2020-03-02.ft')
# gameAll.query('PlayerName == "lengqing" & BetTime >= "2020-02-23 15:50:48"').NetAmount.sum()
# gameAll.query('PlayerName == "lengqing" & BetTime < "2020-02-23 15:50:48" & PayoutTime >= "2020-02-23 15:50:48"').NetAmount.sum()
# MemberAccount.query('MemberName == "lengqing" & AddTime >= "2020-02-23 15:50:48" & (isMainTo == "aggame" | isToMain == "aggame")').Amount.sum()
# member_profile.query('member_name == "lengqing"')
# MemberWallets.query(' MemberId == 235528').sort_values(by = 'Balance2')

In [33]:
# 中心錢包變動紀錄
MemberAccount = pd.DataFrame()
MemberAccountList = os.listdir(pathMemberAccountLog)
for name in MemberAccountList:
    if '.ft' in name:        
        # 2020年02月後才需要，地方錢包金額是從 2020/2/19 之後開始打
        if (int(name[-5:-3]) > 1) & (int(name[-9:-5]) >= 2020):
            MemberAccount = MemberAccount.append(ft.read_dataframe(pathMemberAccountLog + name))
MemberAccount = MemberAccount.query('AddTime < @SpecifyDate')
            

In [34]:
# 遊戲代號資料
query = ''' SELECT ProductId, Name
            FROM [LckBaseDB].[dbo].[Product] with(nolock) '''
productId = utils.read_sql('IDC_ALL', 'LckBaseDB', query)
productId = productId.query('~(ProductId.isin([4, 6, 8, 10, 11, 13, 16, 18, 24, 30]))')
# 技術債，過去的命名方式有與遊戲名稱並不統一
productId['lowerName'] = productId.Name.apply(lambda x:x.lower() if x != 'PG' else x)
productId.loc[productId.lowerName == '188casino', 'lowerName'] = 'grandsuite'
productId.loc[productId.lowerName == 'lbkeno', 'lowerName'] = 'keno'
productId.loc[productId.lowerName == 'mscasino', 'lowerName'] = 'msclub'
productId.loc[productId.lowerName == '188sport', 'lowerName'] = 'sportsbook'

In [10]:
def rawDataAppendNewRegMember(member_profile, MemberWallets, productId, SpecifyDate):
    
    #  要補的用戶名單
    allMember = list(MemberWallets.MemberId.unique())
    lossMember = set(member_profile.query('reg_date >= "2020-02-15" & reg_date < @SpecifyDate').member_id) - set(allMember)
    
    # 補上所有 ProductId 的資料
    appendDf = pd.DataFrame()
    allProductId = set(productId.ProductId) - set([19])
    allProductIdLen = len(allProductId)
    for ProductId in allProductId:
        appendDf = appendDf.append(
            pd.DataFrame({'MemberId':list(lossMember),
                  'ProductId':np.repeat(ProductId, len(lossMember)),
                  'CreateTime':np.repeat(pd.datetime.strptime("2020-02-15 00:00:00", '%Y-%m-%d %H:%M:%S'), len(lossMember)),
                  'UpdateTime':np.repeat(pd.datetime.strptime("2020-02-15 00:00:00", '%Y-%m-%d %H:%M:%S'), len(lossMember)),
                  'Balance':np.repeat(0, len(lossMember))
                 })
        )
        
    # 合併資料
    MemberWallets = MemberWallets.append(appendDf)
    
    return MemberWallets.reset_index(drop=True)

In [11]:
def adjustIMSportWallet(SpecifyDate):
    
    member_profile = ft.read_dataframe('../../share/data_prod/' + 'member_profile.ft')
    MemberWallets = ft.read_dataframe('../../share/data_prod/MemberWallets/' + 'MemberWalletsFlow' + SpecifyDate.replace('-', '') + '.ft')

    # 撈取 IM 資料    
    IMBetRecord = pd.DataFrame()

    startList = ['2020-02-15', '2020-02-18', '2020-02-21', '2020-02-24', '2020-02-28', '2020-03-02', '2020-03-06']
    endList = ['2020-02-18', '2020-02-21', '2020-02-24', '2020-02-28', '2020-03-02', '2020-03-06', '2020-03-07']

    for start, end in zip(startList, endList):

        query = ''' SELECT [BetTime]
                          ,[PlayerName]
                          ,[BetAmount]
                          ,[NetAmount]
                          ,[PayoutTime]
                      FROM [dbo].[IMSportBetRecord] with(nolock)
                      where BetTime between '{}' and '{}'
                        and NetAmount = 0
                        and IsSettled = 1  '''.format(start, end)
        IMBetRecord = IMBetRecord.append(utils.read_sql('New', 'dbFetchStakeMain', query))


    # 需要 MemberId 來與 MemberWallets 合併，取得 地方錢包餘額 的呼叫時間
    IMBetRecord['lowerName'] = IMBetRecord.PlayerName.str.lower()
    IMBetRecord = IMBetRecord.merge(member_profile[['member_name', 'member_id']].rename(columns={'member_name':'lowerName', 'member_id':'MemberId'}), on = 'lowerName', how = 'left')
    IMBetRecord = IMBetRecord.merge(MemberWallets.query('ProductId == 17')[['MemberId', 'CreateTime']], on = 'MemberId', how = 'left')
    
    # 有些用戶並沒有在名單內（1年未上線回鍋不在要計算地方錢包餘額的名單內）
    IMBetRecord = IMBetRecord[~IMBetRecord.CreateTime.isna()]
    
    # 調整時差至 UTC+8
    IMBetRecord['BetTime'] += pd.Timedelta('8 hours')
    IMBetRecord['PayoutTime'] += pd.Timedelta('8 hours')
    
    # 需要的資料（取地方錢包餘額前用戶下注，取完後和局或取消局退回流水的資料）
    IMBetRecordNeedAdd = IMBetRecord.query('CreateTime >= BetTime & CreateTime < PayoutTime').groupby('MemberId').BetAmount.sum().reset_index()
    
    # 更新地方錢包金額
    for index in IMBetRecordNeedAdd.index:
        member = IMBetRecordNeedAdd.loc[index, 'MemberId']
        amount = IMBetRecordNeedAdd.loc[index, 'BetAmount']
        MemberWallets.loc[(MemberWallets.MemberId == member) & (MemberWallets.ProductId == 17), 'Balance'] += amount
        
    # 輸出更新資料
    MemberWallets.reset_index(drop=True).to_feather('../../share/data_prod/MemberWallets/' + 'MemberWalletsFlow' + SpecifyDate.replace('-', '') + '.ft')

In [4]:
def adjustIMSportWallet():
    
    member_profile = ft.read_dataframe('../../share/data_prod/' + 'member_profile.ft')
    MemberWallets = ft.read_dataframe('../../share/data_prod/MemberWallets/' + 'rawMemberWallets.ft')

    # 撈取 IM 資料    
    IMBetRecord = pd.DataFrame()

    startList = ['2020-02-15', '2020-02-18', '2020-02-21', '2020-02-24', '2020-02-28', '2020-03-02', '2020-03-06']
    endList = ['2020-02-18', '2020-02-21', '2020-02-24', '2020-02-28', '2020-03-02', '2020-03-06', '2020-03-07']

    for start, end in zip(startList, endList):

        query = ''' SELECT [BetTime]
                          ,[PlayerName]
                          ,[BetAmount]
                          ,[NetAmount]
                          ,[PayoutTime]
                      FROM [dbo].[IMSportBetRecord] with(nolock)
                      where BetTime between '{}' and '{}'
                        and NetAmount = 0
                        and IsSettled = 1  '''.format(start, end)
        IMBetRecord = IMBetRecord.append(utils.read_sql('New', 'dbFetchStakeMain', query))


    # 需要 MemberId 來與 MemberWallets 合併，取得 地方錢包餘額 的呼叫時間
    IMBetRecord['lowerName'] = IMBetRecord.PlayerName.str.lower()
    IMBetRecord = IMBetRecord.merge(member_profile[['member_name', 'member_id']].rename(columns={'member_name':'lowerName', 'member_id':'MemberId'}), on = 'lowerName', how = 'left')
    IMBetRecord = IMBetRecord.merge(MemberWallets.query('ProductId == 17')[['MemberId', 'CreateTime']], on = 'MemberId', how = 'left')
    
    # 有些用戶並沒有在名單內（1年未上線回鍋不在要計算地方錢包餘額的名單內）
    IMBetRecord = IMBetRecord[~IMBetRecord.CreateTime.isna()]
    
    # 調整時差至 UTC+8
    IMBetRecord['BetTime'] += pd.Timedelta('8 hours')
    IMBetRecord['PayoutTime'] += pd.Timedelta('8 hours')
    
    # 需要的資料（取地方錢包餘額前用戶下注，取完後和局或取消局退回流水的資料）
    IMBetRecordNeedAdd = IMBetRecord.query('CreateTime >= BetTime & CreateTime < PayoutTime').groupby('MemberId').BetAmount.sum().reset_index()
    
    # 更新地方錢包金額
    for index in IMBetRecordNeedAdd.index:
        member = IMBetRecordNeedAdd.loc[index, 'MemberId']
        amount = IMBetRecordNeedAdd.loc[index, 'BetAmount']
        MemberWallets.loc[(MemberWallets.MemberId == member) & (MemberWallets.ProductId == 17), 'Balance'] += amount
        
    # 輸出更新資料
    MemberWallets.reset_index(drop=True).to_feather('../../share/data_prod/MemberWallets/' + 'rawMemberWalletsAdjust.ft')

In [12]:
def calLocalWalletChangeByRawStake(SpecifyDate, pathMemberWallet, MemberWallets, MemberAccount, productId, gameAll):
    
    lastOneDay = pd.datetime.strftime(pd.datetime.strptime(SpecifyDate, '%Y-%m-%d') - pd.Timedelta('1 days'), '%Y-%m-%d')
    
    def choseNotNa(x):
        if x[0]:
            return x[0]
        else:
            return x[1]

    # product 命名 - 遊戲名稱  對照
    mapToGameSummary = {
    'aggame':['AsiaGameAgin', 'AsiaGameHunter', 'AsiaGameXin', 'AsiaGameYoplay'],
    'gdcasino':['GoldDeluxCasino'], 
    'immwgfish':['IMMegaWinFish'], 
    'keno':['LibiancKeno'],
    'grandsuite':['NewCasino188'],
    'sportsbook':['NewSport188'], 
    'owsport':['OWSportNumberGame', 'OWVirtualSport', 'OWSport'],
    'PG':['PGSlot'],
    'blcard':['BLCard'],
    'bwesport':['BWESport'],
    'vrlotto':['VRLotto'],
    'tcglotto':['TCGLotto'],
    'ptslot':['PTSlot'],
    'ppslot':['PPSlot'],
    'msclub':['MSCasino'],
    'keno':['LibiancKeno'],
    'kgkeno':['KGKeno'],
    'imsport':['IMSport', 'IMESport'],
    'imsglotto':['IMSGLotto'],
    'imsgcard':['IMSGCard'],
    'cqslot':['CQSlot'],
    'ebetcasino':['EBETCasino'],
    'hlcard':['HLCard'],
    'imcasino':['IMCasino'],
    'imkycard':['IMKYCard'],
    'imlycard':['IMLYCard'],
    'immtcard':['IMMTCard']
    }

    # 如果新增遊戲，則用假設 product 命名的小寫即微遊戲名稱
    while len(mapToGameSummary.keys()) < productId.shape[0]:
        for gameName in set(productId.lowerName) - set(mapToGameSummary.keys()):
            mapToGameSummary[gameName] = list(productId.loc[productId.lowerName == gameName, 'Name'])

    # 反對照也需要
    invMapToGameSummary = {}
    for key, values in mapToGameSummary.items():
        for value in values:
            invMapToGameSummary[value] = key

    # 遊戲代碼從 dict 調整成 list
    gameTypeName = []
    gameProductType = []
    for game in mapToGameSummary.keys():
        for productType in mapToGameSummary[game]:
            gameTypeName.append(game)
            gameProductType.append(productType)

    productMatch = pd.DataFrame({'lowerName':gameTypeName, 'matchName':gameProductType})
    productMatch = productMatch.merge(productId[['ProductId', 'lowerName']], on = 'lowerName', how = 'left')



    # 調整錢包餘額-------------------------------------------------------------------------------------------
    # 19：IMESport，與 IMSport 共通
    wallet = MemberWallets.query('ProductId != 19')




    # 中心至地方
    MemberAccount['isMainTo'] = MemberAccount['Type'].apply(lambda x:x.replace('main to ', '') if 'main to' in x else 0)
    # 地方至中心
    MemberAccount['isToMain'] = MemberAccount['Type'].apply(lambda x:x.replace(' to main', '') if ' to main' in x else 0)

    # 計算派彩-------------------------------------------------------------------------------------------
    gameAllTarget = gameAll.merge(member_profile[['member_name', 'member_id']].rename(columns={'member_name':'PlayerName', 'member_id':'MemberId'}), on = 'PlayerName', how = 'left')      

    gameAllTarget['lowerName'] = gameAllTarget.ProductType.apply(lambda x:invMapToGameSummary[x])

    productMatchOne = productMatch.groupby(['lowerName', 'ProductId'])['matchName'].count().reset_index()
    del productMatchOne['matchName']
    gameAllTarget = gameAllTarget.merge(productMatchOne, on = 'lowerName', how = 'left')

    where = gameAllTarget['ProductId'] == 19
    gameAllTarget.loc[where, 'lowerName'] = 'imsport'
    gameAllTarget.loc[where, 'ProductId'] = 17


    gameAllTarget = gameAllTarget.merge(wallet[['CreateTime', 'MemberId', 'ProductId']], on = ['MemberId', 'ProductId'], how = 'left')
    # # 用戶贏錢會讓地方錢包增加，因此與 NetAmount 相同，不用轉換

    # type1 事前下注，事後派彩 不論輸贏（流水＋輸贏）（有輸一半的，所以不能定 NetAmount 為 0）
    gameChange = gameAllTarget[~gameAllTarget.CreateTime.isna()].query('(BetTime <= CreateTime & PayoutTime >= CreateTime)').groupby(['ProductId', 'MemberId'])['NetAmount', 'BetAmount'].sum().reset_index()
    gameChange['BetBeforeAndWin'] = gameChange['NetAmount'] + gameChange['BetAmount']
    del gameChange['NetAmount']
    del gameChange['BetAmount']
    # type2 事後下注，事後派彩（輸贏）
    gameChange = gameChange.merge(gameAllTarget[~gameAllTarget.CreateTime.isna()].query('(BetTime >= CreateTime)').groupby(['ProductId', 'MemberId']).NetAmount.sum().reset_index(), on = ['ProductId', 'MemberId'], how = 'outer')
    gameChange.BetBeforeAndWin = gameChange.BetBeforeAndWin.fillna(0)
    gameChange.NetAmount = gameChange.NetAmount.fillna(0)
    gameChange['gross_revenue_bet_time'] = gameChange['BetBeforeAndWin'] + gameChange['NetAmount']

    # 中心、地方轉帳-------------------------------------------------------------------------------------------

    # 只需考慮中心與地方的進出金額
    MemberAccount = MemberAccount.query('isMainTo != 0 | isToMain != 0')
    MemberAccount = MemberAccount.merge(productId[['lowerName', 'ProductId']].rename(columns={'lowerName':'isMainTo', 'ProductId':'isMainToProductId'}),
                          on = 'isMainTo', how = 'left')
    MemberAccount = MemberAccount.merge(productId[['lowerName', 'ProductId']].rename(columns={'lowerName':'isToMain', 'ProductId':'isToMainProductId'}),
                          on = 'isToMain', how = 'left')
    wallet = wallet.merge(productId[['lowerName', 'ProductId']],
                          on = 'ProductId', how = 'left')

    # 創出一行 isToMain + isMainTo 非 NA ，以方便合併 CreateTime
    MemberAccount['lowerName'] = MemberAccount[['isToMain', 'isMainTo']].apply(choseNotNa, axis=1)
    MemberAccount = MemberAccount.merge(wallet[['MemberId', 'lowerName', 'CreateTime']],
                          on = ['MemberId', 'lowerName'], how = 'left')
    MemberAccount = MemberAccount.query('AddTime >= CreateTime')

    # 分別計算 中心轉地方金額、地方轉中心金額
    toProduct = MemberAccount.query('isMainTo != 0').groupby(['isMainToProductId', 'MemberId'])['Amount'].sum().reset_index()
    toProduct.Amount = toProduct.Amount * -1
    toMain = MemberAccount.query('isToMain != 0').groupby(['isToMainProductId', 'MemberId'])['Amount'].sum().reset_index()
    toMain.Amount = toMain.Amount * -1

    # 合併 輸贏＋地方錢包進出的變化
    gameChange = gameChange.merge(toProduct, left_on = ['ProductId', 'MemberId'], right_on = ['isMainToProductId', 'MemberId'], how = 'outer')
    gameChange = gameChange.merge(toMain, left_on = ['ProductId', 'MemberId'], right_on = ['isToMainProductId', 'MemberId'], how = 'outer')
    gameChange = gameChange.fillna(0)
    gameChange['changeTotal'] = gameChange.gross_revenue_bet_time + gameChange.Amount_x + gameChange.Amount_y

    # 會有存款但沒有投注的問題，故要補上 ProductId
    def choseNotZero(x):
        if x[0] != 0:
            return x[0]
        elif x[1] != 0:
            return x[1]
        else:
            pass

    gameChange.loc[gameChange.ProductId == 0, 'ProductId'] = gameChange[gameChange.ProductId == 0][['isMainToProductId', 'isToMainProductId']].apply(choseNotZero, axis=1)
    gameChange = gameChange.groupby(['ProductId', 'MemberId'])['BetBeforeAndWin', 'NetAmount',
                                                               'gross_revenue_bet_time', 'isMainToProductId', 'Amount_x',
                                                               'isToMainProductId', 'Amount_y', 'changeTotal'].agg({'BetBeforeAndWin':'sum', 'NetAmount':'sum',
                                                               'gross_revenue_bet_time':'sum', 'isMainToProductId':'min', 'Amount_x':'sum',
                                                               'isToMainProductId':'min', 'Amount_y':'sum', 'changeTotal':'sum'}).reset_index()
    # 最後的計算處理
    wallet = wallet.merge(gameChange[['MemberId', 'changeTotal', 'ProductId']], on = ['MemberId', 'ProductId'], how = 'left')
    # 處理有些沒有異動的地方錢包
    wallet['changeTotal'] = wallet['changeTotal'].fillna(0)
    wallet['Balance2'] = wallet['Balance'] + wallet['changeTotal']
    wallet['BalanceUpDateTime'] = pd.datetime.strptime(lastOneDay + ' 23:59:59', '%Y-%m-%d %H:%M:%S')

    wallet.reset_index(drop=True).to_feather(pathMemberWallet + 'MemberWalletsFlow' + SpecifyDate.replace('-', '') + '.ft')
    return wallet, gameChange

In [5]:
# MemberWallets = rawDataAppendNewRegMember(member_profile, MemberWallets, productId, SpecifyDate)
# adjustIMSportWallet(SpecifyDate)

# wallet, gameChange = calLocalWalletChangeByRawStake(SpecifyDate, pathMemberWallet, MemberWallets, MemberAccount, productId, gameAll)


In [ ]:
adjustIMSportWallet()

In [35]:
MemberWallets = ft.read_dataframe(pathMemberWallet + 'rawMemberWalletsAdjust.ft')
MemberWallets = rawDataAppendNewRegMember(member_profile, MemberWallets, productId, SpecifyDate)

In [36]:
lastOneDay = pd.datetime.strftime(pd.datetime.strptime(SpecifyDate, '%Y-%m-%d') - pd.Timedelta('1 days'), '%Y-%m-%d')

def choseNotNa(x):
    if x[0]:
        return x[0]
    else:
        return x[1]

# product 命名 - 遊戲名稱  對照
mapToGameSummary = {
'aggame':['AsiaGameAgin', 'AsiaGameHunter', 'AsiaGameXin', 'AsiaGameYoplay'],
'gdcasino':['GoldDeluxCasino'], 
'immwgfish':['IMMegaWinFish'], 
'keno':['LibiancKeno'],
'grandsuite':['NewCasino188'],
'sportsbook':['NewSport188'], 
'owsport':['OWSportNumberGame', 'OWVirtualSport', 'OWSport'],
'PG':['PGSlot'],
'blcard':['BLCard'],
'bwesport':['BWESport'],
'vrlotto':['VRLotto'],
'tcglotto':['TCGLotto'],
'ptslot':['PTSlot'],
'ppslot':['PPSlot'],
'msclub':['MSCasino'],
'keno':['LibiancKeno'],
'kgkeno':['KGKeno'],
'imsport':['IMSport', 'IMESport'],
'imsglotto':['IMSGLotto'],
'imsgcard':['IMSGCard'],
'cqslot':['CQSlot'],
'ebetcasino':['EBETCasino'],
'hlcard':['HLCard'],
'imcasino':['IMCasino'],
'imkycard':['IMKYCard'],
'imlycard':['IMLYCard'],
'immtcard':['IMMTCard']
}

# 如果新增遊戲，則用假設 product 命名的小寫即微遊戲名稱
while len(mapToGameSummary.keys()) < productId.shape[0]:
    for gameName in set(productId.lowerName) - set(mapToGameSummary.keys()):
        mapToGameSummary[gameName] = list(productId.loc[productId.lowerName == gameName, 'Name'])

# 反對照也需要
invMapToGameSummary = {}
for key, values in mapToGameSummary.items():
    for value in values:
        invMapToGameSummary[value] = key

# 遊戲代碼從 dict 調整成 list
gameTypeName = []
gameProductType = []
for game in mapToGameSummary.keys():
    for productType in mapToGameSummary[game]:
        gameTypeName.append(game)
        gameProductType.append(productType)

productMatch = pd.DataFrame({'lowerName':gameTypeName, 'matchName':gameProductType})
productMatch = productMatch.merge(productId[['ProductId', 'lowerName']], on = 'lowerName', how = 'left')



# 調整錢包餘額-------------------------------------------------------------------------------------------
# 19：IMESport，與 IMSport 共通
wallet = MemberWallets.query('ProductId != 19')




# 中心至地方
MemberAccount['isMainTo'] = MemberAccount['Type'].apply(lambda x:x.replace('main to ', '') if 'main to' in x else 0)
# 地方至中心
MemberAccount['isToMain'] = MemberAccount['Type'].apply(lambda x:x.replace(' to main', '') if ' to main' in x else 0)

# 計算派彩-------------------------------------------------------------------------------------------
gameAllTarget = gameAll.merge(member_profile[['member_name', 'member_id']].rename(columns={'member_name':'PlayerName', 'member_id':'MemberId'}), on = 'PlayerName', how = 'left')      

gameAllTarget['lowerName'] = gameAllTarget.ProductType.apply(lambda x:invMapToGameSummary[x])

productMatchOne = productMatch.groupby(['lowerName', 'ProductId'])['matchName'].count().reset_index()
del productMatchOne['matchName']
gameAllTarget = gameAllTarget.merge(productMatchOne, on = 'lowerName', how = 'left')

where = gameAllTarget['ProductId'] == 19
gameAllTarget.loc[where, 'lowerName'] = 'imsport'
gameAllTarget.loc[where, 'ProductId'] = 17


gameAllTarget = gameAllTarget.merge(wallet[['CreateTime', 'MemberId', 'ProductId']], on = ['MemberId', 'ProductId'], how = 'left')
# # 用戶贏錢會讓地方錢包增加，因此與 NetAmount 相同，不用轉換

# type1 事前下注，事後派彩 不論輸贏（流水＋輸贏）（有輸一半的，所以不能定 NetAmount 為 0）
gameChange = gameAllTarget[~gameAllTarget.CreateTime.isna()].query('(BetTime <= CreateTime & PayoutTime >= CreateTime)').groupby(['ProductId', 'MemberId'])['NetAmount', 'BetAmount'].sum().reset_index()
gameChange['BetBeforeAndWin'] = gameChange['NetAmount'] + gameChange['BetAmount']
del gameChange['NetAmount']
del gameChange['BetAmount']
# type2 事後下注，事後派彩（輸贏）
gameChange = gameChange.merge(gameAllTarget[~gameAllTarget.CreateTime.isna()].query('(BetTime >= CreateTime)').groupby(['ProductId', 'MemberId']).NetAmount.sum().reset_index(), on = ['ProductId', 'MemberId'], how = 'outer')
gameChange.BetBeforeAndWin = gameChange.BetBeforeAndWin.fillna(0)
gameChange.NetAmount = gameChange.NetAmount.fillna(0)
gameChange['gross_revenue_bet_time'] = gameChange['BetBeforeAndWin'] + gameChange['NetAmount']

# 中心、地方轉帳-------------------------------------------------------------------------------------------

# 只需考慮中心與地方的進出金額
MemberAccount = MemberAccount.query('isMainTo != 0 | isToMain != 0')
MemberAccount = MemberAccount.merge(productId[['lowerName', 'ProductId']].rename(columns={'lowerName':'isMainTo', 'ProductId':'isMainToProductId'}),
                      on = 'isMainTo', how = 'left')
MemberAccount = MemberAccount.merge(productId[['lowerName', 'ProductId']].rename(columns={'lowerName':'isToMain', 'ProductId':'isToMainProductId'}),
                      on = 'isToMain', how = 'left')
wallet = wallet.merge(productId[['lowerName', 'ProductId']],
                      on = 'ProductId', how = 'left')

# 創出一行 isToMain + isMainTo 非 NA ，以方便合併 CreateTime
MemberAccount['lowerName'] = MemberAccount[['isToMain', 'isMainTo']].apply(choseNotNa, axis=1)
MemberAccount = MemberAccount.merge(wallet[['MemberId', 'lowerName', 'CreateTime']],
                      on = ['MemberId', 'lowerName'], how = 'left')
MemberAccount = MemberAccount.query('AddTime >= CreateTime')

# 分別計算 中心轉地方金額、地方轉中心金額
toProduct = MemberAccount.query('isMainTo != 0').groupby(['isMainToProductId', 'MemberId'])['Amount'].sum().reset_index()
toProduct.Amount = toProduct.Amount * -1
toMain = MemberAccount.query('isToMain != 0').groupby(['isToMainProductId', 'MemberId'])['Amount'].sum().reset_index()
toMain.Amount = toMain.Amount * -1

# 合併 輸贏＋地方錢包進出的變化
gameChange = gameChange.merge(toProduct, left_on = ['ProductId', 'MemberId'], right_on = ['isMainToProductId', 'MemberId'], how = 'outer')
gameChange = gameChange.merge(toMain, left_on = ['ProductId', 'MemberId'], right_on = ['isToMainProductId', 'MemberId'], how = 'outer')
gameChange = gameChange.fillna(0)
gameChange['changeTotal'] = gameChange.gross_revenue_bet_time + gameChange.Amount_x + gameChange.Amount_y

# 會有存款但沒有投注的問題，故要補上 ProductId
def choseNotZero(x):
    if x[0] != 0:
        return x[0]
    elif x[1] != 0:
        return x[1]
    else:
        pass

gameChange.loc[gameChange.ProductId == 0, 'ProductId'] = gameChange[gameChange.ProductId == 0][['isMainToProductId', 'isToMainProductId']].apply(choseNotZero, axis=1)
gameChange = gameChange.groupby(['ProductId', 'MemberId'])['BetBeforeAndWin', 'NetAmount',
                                                           'gross_revenue_bet_time', 'isMainToProductId', 'Amount_x',
                                                           'isToMainProductId', 'Amount_y', 'changeTotal'].agg({'BetBeforeAndWin':'sum', 'NetAmount':'sum',
                                                           'gross_revenue_bet_time':'sum', 'isMainToProductId':'min', 'Amount_x':'sum',
                                                           'isToMainProductId':'min', 'Amount_y':'sum', 'changeTotal':'sum'}).reset_index()
# 最後的計算處理
wallet = wallet.merge(gameChange[['MemberId', 'changeTotal', 'ProductId']], on = ['MemberId', 'ProductId'], how = 'left')
# 處理有些沒有異動的地方錢包
wallet['changeTotal'] = wallet['changeTotal'].fillna(0)
wallet['Balance2'] = wallet['Balance'] + wallet['changeTotal']
wallet['BalanceUpDateTime'] = pd.datetime.strptime(lastOneDay + ' 23:59:59', '%Y-%m-%d %H:%M:%S')

In [47]:
gameAll2 = gameAll.merge(member_profile[['member_name', 'member_id']].rename(columns={'member_name':'PlayerName', 'member_id':'MemberId'}), on = 'PlayerName', how = 'left')

In [58]:
wallet.query('Balance2 < 0 & Balance >= 0 & ProductId != 14').sort_values(by = 'Balance2').head(100)

,Balance,CreateTime,Id,MemberId,ProductId,UpdateTime,lowerName,changeTotal,Balance2,BalanceUpDateTime
595551,20513.67,2020-02-25 01:12:32.870,620555.0,73123,5,2020-02-25 01:12:32.870,msclub,-40513.420000,-19999.750000,2020-03-06 23:59:59
814450,0.55,2020-02-26 17:17:10.207,848633.0,491374,7,2020-02-26 17:17:10.207,aggame,-6750.400000,-6749.850000,2020-03-06 23:59:59
445086,4400.00,2020-02-24 00:13:45.733,463771.0,156895,7,2020-02-24 00:13:45.733,aggame,-10027.950000,-5627.950000,2020-03-06 23:59:59
41664,23600.00,2020-02-21 16:07:06.440,43419.0,109778,2,2020-02-21 16:07:06.440,sportsbook,-28253.100000,-4653.100000,2020-03-06 23:59:59
176714,0.50,2020-02-22 08:48:48.853,184155.0,232615,22,2020-02-22 08:48:48.853,imlycard,-4082.650000,-4082.150000,2020-03-06 23:59:59
194376,0.00,2020-02-22 11:02:42.877,202557.0,112097,2,2020-02-22 11:02:42.877,sportsbook,-3659.150000,-3659.150000,2020-03-06 23:59:59
368705,20810.39,2020-02-23 11:50:18.620,384188.0,144754,7,2020-02-23 11:50:18.620,aggame,-23809.400000,-2999.010000,2020-03-06 23:59:59
59636,675.30,2020-02-21 18:09:12.920,62157.0,465479,26,2020-02-21 18:09:12.920,immtcard,-3477.670000,-2802.370000,2020-03-06 23:59:59
603883,0.02,2020-02-25 02:42:35.863,629239.0,464884,25,2020-02-25 02:42:35.863,owsport,-2573.070000,-2573.050000,2020-03-06 23:59:59
414888,0.43,2020-02-23 19:21:06.323,432309.0,440367,17,2020-02-23 19:21:06.323,imsport,-2168.315693,-2167.885693,2020-03-06 23:59:59


檢查問題

In [201]:
index = 814450
Balance, CreateTime, _, MemberId, ProductId, _, lowerName, _, _, _ = wallet.loc[index, :]
ProductType = mapToGameSummary[lowerName]
Balance, CreateTime

(0.55, Timestamp('2020-02-26 17:17:10.207000'))

In [202]:
gameTmp = gameAll2.query('MemberId == @MemberId & ProductType == @ProductType & (PayoutTime >= @CreateTime & PayoutTime < @SpecifyDate)').sort_values(by = 'PayoutTime').reset_index(drop=True)
# 事前下注對錢包的影響是 流水＋輸贏
gameTmp.loc[(gameTmp.BetTime <= CreateTime), 'NetAmount'] += gameTmp.loc[(gameTmp.BetTime <= CreateTime), 'BetAmount'] 
gameTmp['cum'] = np.cumsum(gameTmp.NetAmount)
gameTmp['Type'] = 'BetRecord'

In [203]:
member.query('MemberId == 491374')

,MemberId,MemberName,o_member_name,Status,FirstName,LastName,MobilePhone,PaymentStatus,Affiliate,IsEmail,...,RegDate,SeniorMember,IsOpenLargeCustomers,WithdrawalStatus,Source,IsDepositActive,IsWithdrawalActive,IsTransferActive,IsSmsNotifyActive,IsPushNotifyActive
489533,491374,zhang456789,zhang456789,1,张智文,,14780196231,1,0,0,...,2019-10-10 00:21:22.463,False,False,True,0,True,True,True,False,True


In [204]:
gameTmp.head(1)

,NetAmount,PlayerName,PayoutTime,BetTime,BetAmount,ProductType,MemberId,cum,Type
0,-50.0,zhang456789,2020-02-26 23:44:21,2020-02-26 23:44:04,50.0,AsiaGameYoplay,491374,-50.0,BetRecord


In [205]:
gameChange.query('MemberId == @MemberId')

,ProductId,MemberId,BetBeforeAndWin,NetAmount,gross_revenue_bet_time,isMainToProductId,Amount_x,isToMainProductId,Amount_y,changeTotal
7244,7.0,491374,0.0,45059.6,45059.6,7.0,367000.0,7.0,-418810.0,-6750.4
8310,9.0,491374,0.0,0.0,0.0,0.0,9500.0,0.0,-9500.0,0.0


In [206]:
MemberAccountFull = ft.read_dataframe('../../share/data_prod/MemberAccountLog/MemberAccountLog202002.ft')
MemberAccountFull = MemberAccountFull.append(ft.read_dataframe('../../share/data_prod/MemberAccountLog/MemberAccountLog202003.ft')).reset_index(drop=True).query('AddTime < "2020-03-02"')

In [207]:
checkLoss = MemberAccountFull.query('MemberId == @MemberId').sort_values(by = 'AddTime').reset_index(drop = True)

In [208]:
OldBalance = checkLoss.loc[0, 'CurrentBalance']
for i in range(1, checkLoss.shape[0]):
    CurrentBalance = checkLoss.loc[i, 'OldBalance']
    if CurrentBalance!=OldBalance:
        print(i)
    OldBalance = checkLoss.loc[i, 'CurrentBalance']

In [209]:
MemberAccountTmp = MemberAccount.query('MemberId == @MemberId & ((isMainToProductId == @ProductId) | (isToMainProductId == @ProductId)) & (AddTime >= @CreateTime)').sort_values(by = 'AddTime').reset_index(drop=True)
# MemberAccount.query('MemberId == @MemberId & AddTime >= @CreateTime').sort_values(by = 'AddTime').reset_index(drop=True)
MemberAccountTmp['changeFlow'] = np.cumsum(MemberAccountTmp.Amount) * -1
MemberAccountTmp['Type'] = 'Account'

In [210]:
concatAll = gameTmp[['PayoutTime', 'NetAmount', 'Type']].rename(columns={'PayoutTime':'AddTime', 'NetAmount':'Amount'})
concatAll = concatAll.append(MemberAccountTmp[['AddTime', 'Amount', 'Type']]).sort_values(by = 'AddTime').reset_index(drop=True)
concatAll.loc[concatAll.Type == "Account", 'Amount'] *= -1
concatAll['changeFlow'] = np.cumsum(concatAll.Amount)
concatAll['gassFlow'] = concatAll.changeFlow + Balance

In [211]:
pd.options.display.max_rows = 5000
Balance, CreateTime, lowerName, gameTmp.loc[0, 'PlayerName']

(0.55, Timestamp('2020-02-26 17:17:10.207000'), 'aggame', 'zhang456789')

In [212]:
concatAll

,AddTime,Amount,Type,changeFlow,gassFlow
0,2020-02-26 23:43:49.397,9500.0,Account,9500.0,9500.55
1,2020-02-26 23:44:21.000,-50.0,BetRecord,9450.0,9450.55
2,2020-02-26 23:44:21.000,-50.0,BetRecord,9400.0,9400.55
3,2020-02-26 23:44:21.000,-50.0,BetRecord,9350.0,9350.55
4,2020-02-26 23:44:21.000,-50.0,BetRecord,9300.0,9300.55
...,...,...,...,...,...
15293,2020-03-06 17:05:57.000,-2.5,BetRecord,36256.6,36257.15
15294,2020-03-06 17:06:00.000,-2.5,BetRecord,36254.1,36254.65
15295,2020-03-06 17:06:01.000,-2.0,BetRecord,36252.1,36252.65
15296,2020-03-06 17:06:06.000,-2.5,BetRecord,36249.6,36250.15


In [169]:
target = "2020-03-06 02:08:56"
target = pd.datetime.strftime(pd.datetime.strptime(target, '%Y-%m-%d %H:%M:%S') + pd.Timedelta('+12 hours'), '%Y-%m-%d %H:%M:%S')
targetEnd = target + '.999'

已誤差970

In [170]:
concatAll.query('AddTime >= @target & AddTime < @targetEnd')

,AddTime,Amount,Type,changeFlow,gassFlow
14115,2020-03-06 14:08:56.367,9500.0,Account,2749.75,2750.3


In [152]:
18410.95 - 970 

17440.95

In [150]:
(4551.25+250) / 4800

1.0002604166666667

In [129]:
target

'2020-02-28 02:36:53'

In [132]:
gameTmp.query('BetTime >= "2020-02-28 02:30:53" & BetTime < @targetEnd')

,NetAmount,PlayerName,PayoutTime,BetTime,BetAmount,ProductType,MemberId,cum,Type
646,-100.00,zhang456789,2020-02-28 02:31:18,2020-02-28 02:31:04,100.0,AsiaGameYoplay,491374,10050.00,BetRecord
647,-50.00,zhang456789,2020-02-28 02:31:18,2020-02-28 02:31:04,50.0,AsiaGameYoplay,491374,10000.00,BetRecord
648,-50.00,zhang456789,2020-02-28 02:31:18,2020-02-28 02:31:04,50.0,AsiaGameYoplay,491374,9950.00,BetRecord
649,-50.00,zhang456789,2020-02-28 02:31:52,2020-02-28 02:31:37,50.0,AsiaGameYoplay,491374,9900.00,BetRecord
650,-100.00,zhang456789,2020-02-28 02:31:52,2020-02-28 02:31:35,100.0,AsiaGameYoplay,491374,9800.00,BetRecord
651,-50.00,zhang456789,2020-02-28 02:31:52,2020-02-28 02:31:35,50.0,AsiaGameYoplay,491374,9750.00,BetRecord
652,-50.00,zhang456789,2020-02-28 02:31:52,2020-02-28 02:31:35,50.0,AsiaGameYoplay,491374,9700.00,BetRecord
653,-100.00,zhang456789,2020-02-28 02:32:26,2020-02-28 02:32:14,100.0,AsiaGameYoplay,491374,9600.00,BetRecord
654,-100.00,zhang456789,2020-02-28 02:32:26,2020-02-28 02:32:14,100.0,AsiaGameYoplay,491374,9500.00,BetRecord
655,-150.00,zhang456789,2020-02-28 02:33:00,2020-02-28 02:32:47,150.0,AsiaGameYoplay,491374,9350.00,BetRecord


In [130]:
concatAll.query('AddTime >= "2020-02-28 02:30:53" & AddTime < @targetEnd')

,AddTime,Amount,Type,changeFlow,gassFlow
654,2020-02-28 02:31:18,-100.00,BetRecord,15350.00,15350.55
655,2020-02-28 02:31:18,-50.00,BetRecord,15300.00,15300.55
656,2020-02-28 02:31:18,-50.00,BetRecord,15250.00,15250.55
657,2020-02-28 02:31:52,-50.00,BetRecord,15200.00,15200.55
658,2020-02-28 02:31:52,-100.00,BetRecord,15100.00,15100.55
659,2020-02-28 02:31:52,-50.00,BetRecord,15050.00,15050.55
660,2020-02-28 02:31:52,-50.00,BetRecord,15000.00,15000.55
661,2020-02-28 02:32:26,-100.00,BetRecord,14900.00,14900.55
662,2020-02-28 02:32:26,-100.00,BetRecord,14800.00,14800.55
663,2020-02-28 02:33:00,-150.00,BetRecord,14650.00,14650.55


In [112]:
targetEnd

'2020-02-27 02:27:20.999'

In [131]:
-149.25/150

-0.995

觀察錯誤

In [72]:
index = 445066
_, CreateTime, _, MemberId, ProductId, _, _, _, _, _, = wallet.loc[index, :]

In [83]:
PlayerName

'lsz1234'

In [73]:
PlayerName = member_profile.query('member_id == @MemberId').reset_index(drop=True).member_name[0]

In [74]:
ProductType = mapToGameSummary[productId.query('ProductId == @ProductId').reset_index().lowerName[0]][0]

之前下注、之後派彩

In [75]:
gameAll.query('PlayerName == @PlayerName & BetTime < @CreateTime & PayoutTime >= @CreateTime  & PayoutTime < "2020-03-02" & ProductType == @ProductType').shape[0], gameAll.query('PlayerName == @PlayerName & BetTime < @CreateTime & PayoutTime >= @CreateTime  & PayoutTime < "2020-03-02" & ProductType == @ProductType').NetAmount.sum()

(0, 0.0)

In [84]:
PlayerName

'lsz1234'

In [86]:
gameAll.query('PlayerName == @PlayerName & BetTime >= @CreateTime').head()

,NetAmount,PlayerName,PayoutTime,BetTime,BetAmount,ProductType
19850385,5000.0,lsz1234,2020-02-24 00:15:19,2020-02-24 00:14:51,5000.0,AsiaGameAgin
19851026,-2000.0,lsz1234,2020-02-24 00:16:03,2020-02-24 00:15:47,2000.0,AsiaGameAgin
19851732,-3300.0,lsz1234,2020-02-24 00:16:43,2020-02-24 00:16:30,3300.0,AsiaGameAgin
19852191,3990.0,lsz1234,2020-02-24 00:17:28,2020-02-24 00:17:09,3990.0,AsiaGameAgin
19852939,2000.0,lsz1234,2020-02-24 00:18:11,2020-02-24 00:17:58,2000.0,AsiaGameAgin


In [88]:
gameAll.query('PlayerName == @PlayerName & BetTime < @CreateTime').tail()

,NetAmount,PlayerName,PayoutTime,BetTime,BetAmount,ProductType
19888846,1567.5,lsz1234,2020-02-24 00:08:11,2020-02-24 00:07:53,1567.5,AsiaGameAgin
19891729,1995.0,lsz1234,2020-02-24 00:08:48,2020-02-24 00:08:34,1995.0,AsiaGameAgin
19891767,-1330.0,lsz1234,2020-02-24 00:09:26,2020-02-24 00:09:11,1330.0,AsiaGameAgin
19891811,-2300.0,lsz1234,2020-02-24 00:10:04,2020-02-24 00:09:51,2300.0,AsiaGameAgin
19892792,475.0,lsz1234,2020-02-24 00:10:40,2020-02-24 00:10:31,475.0,AsiaGameAgin


一班下注

In [76]:
gameAll.query('PlayerName == @PlayerName & BetTime > @CreateTime & PayoutTime < "2020-03-02" & ProductType == @ProductType').shape[0], gameAll.query('PlayerName == @PlayerName & BetTime > @CreateTime & PayoutTime < "2020-03-02" & ProductType == @ProductType').NetAmount.sum()

(341, 35051.049999999996)

輸贏＋錢包變化統計

In [77]:
gameChange.query(' MemberId == @MemberId')

,ProductId,MemberId,BetBeforeAndWin,NetAmount,gross_revenue_bet_time,isMainToProductId,Amount_x,isToMainProductId,Amount_y,changeTotal
5286,7.0,156895,0.0,35051.05,35051.05,7.0,156931.0,7.0,-202010.0,-10027.95


錢包變化

In [79]:
MemberAccount.query('MemberId == @MemberId & AddTime >= @CreateTime & AddTime < "2020-03-02"').sort_values(by = 'AddTime')

,LId,MemberId,MemberName,OrderId,Currency,CurrentBalance,OldBalance,Amount,Type,AddTime,Remark,isMainTo,isToMain
107181,44484602,156895,lsz1234,2002240032436106362,RMB,25690.40,0.40,25690.00,aggame to main,2020-02-24 00:32:33.877,aggame to main,0,aggame
107438,44484772,156895,lsz1234,PTD2002240033gbyo6,RMB,0.40,25690.40,-25690.00,Payment-Withdrawal,2020-02-24 00:39:10.620,,0,0
113642,44499114,156895,lsz1234,BA2002241456othah,RMB,1311.14,0.40,1310.74,payment-batchAdjuest,2020-02-24 14:56:44.893,,0,0
731212,44525767,156895,lsz1234,PDB2002251312s5z44,RMB,11011.14,1311.14,9700.00,payment to deposit,2020-02-25 13:14:29.437,,0,0
731229,44525784,156895,lsz1234,2002251315377583485,RMB,11.14,11011.14,-11000.00,main to aggame,2020-02-25 13:15:36.170,main to aggame,aggame,0
731533,44526088,156895,lsz1234,PDB2002251329jbpu5,RMB,6011.14,11.14,6000.00,payment to deposit,2020-02-25 13:31:43.590,,0,0
731542,44526097,156895,lsz1234,2002251332032617316,RMB,11.14,6011.14,-6000.00,main to aggame,2020-02-25 13:32:01.590,main to aggame,aggame,0
732938,44527493,156895,lsz1234,2002251434439353953,RMB,28034.14,11.14,28023.00,aggame to main,2020-02-25 14:34:43.880,aggame to main,0,aggame
733004,44527559,156895,lsz1234,PTD2002251435ud1hj,RMB,0.14,28034.14,-28034.00,Payment-Withdrawal,2020-02-25 14:38:17.250,,0,0
735934,44530489,156895,lsz1234,PDB2002251548j2u5e,RMB,12000.14,0.14,12000.00,payment to deposit,2020-02-25 15:52:28.490,,0,0
